# Required assignment 16.1: Backpropagation using PyTorch

In this activity, you explore the fundamental concepts of backpropagation, a core technique used in training neural networks. At its heart, deep learning is an optimisation problem where you adjust the network's weights to minimise a loss function that measures how well the network’s predictions match the target values.

To effectively minimise this loss, you rely on calculus, specifically the computation of gradients (partial derivatives) of the loss with respect to the weights. Backpropagation efficiently computes these gradients using the chain rule, breaking down complex computations into simpler steps visualised through a computational graph.

You will learn about two modes of differentiation ­– forward mode and backward mode – each suited to different network input–output configurations. Understanding these concepts is critical for implementing and optimising neural networks through algorithms such as stochastic gradient descent.

This activity will guide you through the mechanics of backpropagation and the chain rule, equipping you with a solid foundation for neural network training and optimisation.
To work on this assignment, you require `aima3` and `torchviz`. If these are not installed, please ensure to use `pip install aima3` and `pip install torchviz` to install these packages before importing them.

In [2]:
#import necessary libraries
import sys
from aima3 import learning
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as func
from torch.autograd import Variable
from torchviz import make_dot

### Backpropagation and the chain rule


*Foreword: In this notebook, we use slightly different terminology. An arbitrary training instance is denoted as $(v, y) \in E$ where $v$ is the collection of predictors, $y$ is the target and $E$ is the training set. Moreover, the network weights are denoted by $x$.*

Deep learning is fundamentally a giant problem in optimisation. You are choosing numerical 'weights' to minimise a loss function $L$ (which depends on those weights). **This is the learning part.** In other words,
$$L(x) = \sum_{(v, y) \in \boldsymbol{E}} \text{loss}(F(x, v) - y).$$
Calculus tells us that the minimiser of $L$ satisfies the following system of equations (there may be many solutions that satisfy this, hence you do not necessarily obtain the minimiser – you just hope it's something 'good enough'):

> **The partial derivatives of L with respect to the weights $x$ should be zero**: $$\boxed{\frac{\partial L}{\partial x} = 0 }$$

You solve the equation above, iteratively, using a modification of the gradient descent method called **stochastic gradient descent**.

*Backpropagation* is a method to compute derivatives quickly, using the chain rule:

$$\frac{dF}{dx} = \frac{d}{dx}(F_3(F_2(F_1 (x))) = \frac{dF_3}{dF_2}\vert_{F_2=F_2(F_1(x))} \frac{dF_2(F_1(x))}{dF_1}\vert_{F_1 = F_1(x)} \frac{dF_1(x)}{dx}\vert_x.$$

A convenient way to visualise how the function $F$ is computed from the weights $x_i$ is to use a **computational graph**. It separates the big computation into small steps, and you can find the derivative of each step (each computation) on the graph.

**Backpropagation** is a technique for optimising parameters in a neural network. There are two types of backpropagation, depending on the relation between the number of inputs and outputs in the neural network:
- *Forward mode*: $F$ has few inputs but many outputs
- *Backward mode*: $F$ has many inputs but few outputs

*Forward mode* differentiation tracks how one input affects every node. *Reverse mode* differentiation tracks how every node affects one output. That is, forward mode differentiation applies the operator $\dfrac{\partial (.)}{\partial x}$ to every node, while reverse mode differentiation applies the operator $\dfrac{\partial F}{\partial (.)}$ to every node. The general rule is to sum over all possible paths from one node to the other, multiplying the derivatives on each edge of the path together.

### Question 1:

Consider the computational graph shown below:
<img src="images/image1.png" alt="Drawing" style="width: 300px;"/>

$$
c = x^2
$$

$$
s = x + y
$$

$$
F = c x s
$$
where the initial values of x = 2 and y = 3.

- Compute the derivative $$∂F/∂x$$ using forward-mode differentiation.
- Compute the same derivative $$∂F/∂x$$
  using backward mode automatic differentiation implemented in PyTorch.
- Verify your backward mode derivative by printing the gradient stored in $$x.grad$$


### Question 1a:

Implement the example in PyTorch.
- Provide the inputs `x` and `y` using `Variable(torch.tensor(2.), requires_grad=True)` and `Variable(torch.tensor(3.),requires_grad=True)` respectively.
- From the computation graph given above, input the values of `c`, `s` and `F`.

In [5]:
### GRADED CELL
x = None
y = None
c = None
s = None
F = None
# YOUR CODE HERE
#raise NotImplementedError()

# inputs with gradients enabled
x = Variable(torch.tensor(2.0), requires_grad=True)
y = Variable(torch.tensor(3.0), requires_grad=True)

# computational graph
c = x ** 2      # c = x^2
s = x + y       # s = x + y
F = c * s       # F = c * s

### Question 1b:
Write the PyTorch command that computes the gradients of \( F \) using backward propagation **while keeping the computational graph intact** for further gradient computations.



*Hint:* Use the appropriate argument in the `backward()` method to retain the graph.

In [6]:
###GRADED CELL

# YOUR CODE HERE
# raise NotImplementedError()

# compute gradients of F w.r.t. x and y
F.backward(retain_graph=True)

print(x.grad.numpy())

24.0


### Question 2:

Let $F = \log(x) + x^2 y + y^2$.

Evaluate $\dfrac{\partial{F}}{\partial{x}}$ and $\dfrac{\partial{F}}{\partial{y}}$ at the point $x = 2$, $y = 3$ (both in forward and backward modes)
 by using `torch`.

**Answer:**

You have $\dfrac{\partial F}{\partial x} = 1/x + 2xy$, which evaluates to $12.5$ at $(x=2, y=3)$. Moreover, you also have $\dfrac{\partial F}{\partial y} = x^2 + 2y$, which evaluates to $10$ at $(x=2, y=3)$. To compute this, three ways are demonstrated below.

Now, you can solve this problem simply by using `torch`.

You are given:
$$
F(x, y) = \ln x + x^2y + y^2
$$

You need to:
1. **Break the function into intermediate variables**:  
   Let  
  $$
   a = \ln x,\quad b = x^2y,\quad c = y^2
   $$
   and  
 $$
   F = a + b + c.
   $$

2. **Find the local derivatives** for each intermediate variable with respect to its inputs:  
   

3. **Apply the chain rule** to compute:  
  $$
   \frac{\partial F}{\partial x}
   \quad\text{and}\quad
   \frac{\partial F}{\partial y}.
   $$

4. **Evaluate the derivatives** at:
  $$
   x = 2, y = 3
  $$

5. **Verify the results** using PyTorch – The solved answer in pen and paper is provided above.

###Instructions:
- Assign input variables to `x` and `y` using `Variable(torch.tensor())`. HINT: Refer to Question 1.
- Assign `torch.log(x)` to `a`.
- Refer to the equation and assign value to `b`.
- Refer to the equation and assign value to `c`.
- Compute `F` which is the summation of `a`, `b` and `c`.
- Compute `F.backward()`.
   

In [7]:
###GRADED CELL
x = None
y = None
a = None
b = None
c = None
F = None
# YOUR CODE HERE
# raise NotImplementedError()

# inputs
x = Variable(torch.tensor(2.0), requires_grad=True)
y = Variable(torch.tensor(3.0), requires_grad=True)

# intermediate values
a = torch.log(x)      # a = ln(x)
b = x**2 * y          # b = x^2 * y
c = y**2              # c = y^2

# final expression
F = a + b + c

# compute gradients
F.backward()

print("derivative WRT x:", x.grad.numpy(), "WRT y:", y.grad.numpy())

derivative WRT x: 12.5 WRT y: 10.0


### Stochastic Gradient Descent (SGD)

1. Weight initialization:
the starting weights $$𝑥_0$$
  in gradient descent strongly affect training. Poor initialisation (e.g. all zeros or wrong variance) can cause exploding or vanishing weights, preventing convergence. Layer width and variance together control stability.

2. Gradient descent & SGD:
Gradient descent updates weights iteratively using:

$$
x_{k+1} = x_k - s_k \nabla L(x_k)
$$
SGD speeds up training and improves generalisation by computing gradients on small random mini-batches instead of the entire data set.

3. Practical training stability;
SGD avoids overfitting in practice and benefits from techniques such as momentum, adaptive learning rates (e.g. Adam) and early stopping to converge faster and maintain good performance on unseen data.

### Steps of SGD

1. Mini-batches: random subsets of data are selected in each iteration to compute a noisy estimate of the gradient.

2. Forward pass: compute predictions using current weights.

3. Loss calculation: measure difference between predictions and true labels.

4. Backward pass: use automatic differentiation to find gradients.

5. Weight update: adjust weights incrementally in the opposite direction of gradients.

    Repeat: this process iterates multiple times, progressively reducing the loss and improving model parameters.

In [8]:
dtype = torch.float
device = torch.device("cpu")
B, N, D_in, D_out = 4, 20, 2, 1
xrange = yrange = np.arange(0.0, 1.0, 0.1)
g = np.meshgrid(xrange, yrange, sparse=False, indexing='ij')
_x = np.vstack(tup=tuple(map(np.ravel, g))).T
_w = np.array((0.4, 0.2)).reshape(1, -1).T
eps = 1e-2  #Define eps here
_y = _x.dot(_w) + eps * np.random.rand(_x.shape[0], 1)

#Select a small sample of the data

np.random.seed(42)
idx = np.random.randint(0, 100, N)
x_np = _x[idx]
y_np = _y[idx]

#Create random tensors to hold inputs and outputs.

x = Variable(torch.Tensor(x_np))
y = Variable(torch.Tensor(y_np))

#Create random tensors for weights.
#Setting requires_grad=True indicates that you want to compute gradients with respect to these tensors during the backward pass.
w = torch.randn(D_in, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-2
epochs = 500
weights = np.empty((epochs//10, 2))
losses = np.empty(epochs//10)
for t in range(epochs):
    sample = np.random.randint(0, 20, B)
    x_B, y_B = x[sample], y[sample]
    # Forward pass: compute predicted y using operations on Tensors; 
    y_pred = x_B.mm(w)

    #Compute and print loss using operations on tensors.
    #Now, loss is a tensor of shape (1,)
    #loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y_B).pow(2).mean()

    #Use autograd to compute the backward pass. 
    loss.backward()

    #Manually update weights using gradient descent. Wrap in torch.no_grad().
    
    with torch.no_grad():
        w -= learning_rate * w.grad #This is the gradient of loss with respect to w

        #Manually zero the gradients after updating weights
        w.grad.zero_()

    if t % 10 == 0:
        ind = int(t/10)
        losses[ind] = loss.item()
        weights[ind, :] = w.data.view(1, -1).numpy()[0]
        print(t, losses[ind], weights[ind, :])

#Compare this with the initial weights you had set up in your data.
print('final weights:', w)

0 0.28246793150901794 [-0.81711113  1.2332046 ]
10 0.32800012826919556 [-0.77026129  1.24072444]
20 0.25425708293914795 [-0.73008639  1.25254464]
30 0.015801629051566124 [-0.69650716  1.25042665]
40 0.36563342809677124 [-0.6607812   1.24708831]
50 0.1696881800889969 [-0.63260621  1.24352777]
60 0.18371504545211792 [-0.60611552  1.23942494]
70 0.14574940502643585 [-0.58907056  1.22756052]
80 0.07515283674001694 [-0.56214428  1.2258997 ]
90 0.18182028830051422 [-0.53988492  1.22007561]
100 0.1958242505788803 [-0.51229918  1.20963466]
110 0.13647663593292236 [-0.48752689  1.20617306]
120 0.09315693378448486 [-0.47454092  1.19189417]
130 0.08985738456249237 [-0.4555679  1.1838727]
140 0.015101850964128971 [-0.44640636  1.17761028]
150 0.06805810332298279 [-0.42687866  1.17529058]
160 0.14071322977542877 [-0.40481341  1.16979659]
170 0.022882983088493347 [-0.38662228  1.15833724]
180 0.08650193363428116 [-0.37365228  1.1498785 ]
190 0.1384269744157791 [-0.3633486   1.13890707]
200 0.2201676